In [1]:
import numpy as np

# Data
x = np.array([10.4, 10.8, 11.1, 10.2, 10.3, 10.2, 10.7, 10.5, 10.8, 11.2, 10.6, 11.4])
y = np.array([7.4, 7.6, 7.9, 7.2, 7.4, 7.1, 7.4, 7.2, 7.8, 7.7, 7.8, 8.3])

# Correlation matrix
corr_matrix = np.corrcoef(x, y)

# Extract correlation coefficients
r_xy = corr_matrix[0, 1]
r_yx = corr_matrix[1, 0]

print("Correlation matrix:\n", corr_matrix)
print("R(x,y) =", r_xy)
print("R(y,x) =", r_yx)

Correlation matrix:
 [[1.         0.87035456]
 [0.87035456 1.        ]]
R(x,y) = 0.8703545613058227
R(y,x) = 0.8703545613058227


In [1]:
import numpy as np

# Data
x = np.array([10.4, 10.8, 11.1, 10.2, 10.3, 10.2, 10.7, 10.5, 10.8, 11.2, 10.6, 11.4])
y = np.array([7.4, 7.6, 7.9, 7.2, 7.4, 7.1, 7.4, 7.2, 7.8, 7.7, 7.8, 8.3])

n = len(x)
r = np.corrcoef(x, y)[0, 1]

# Fisher z transform
z = 0.5 * np.log((1 + r) / (1 - r))

# Standard error in z
se_z = 1 / np.sqrt(n - 3)

# 95% CI in z
z_crit = 1.96
z_lower, z_upper = z - z_crit * se_z, z + z_crit * se_z

# Convert back to r
r_lower = (np.exp(2*z_lower) - 1) / (np.exp(2*z_lower) + 1)
r_upper = (np.exp(2*z_upper) - 1) / (np.exp(2*z_upper) + 1)

print("Correlation r =", r)
print("Standard error (z scale) =", se_z)
print("95% CI for r: [{:.3f}, {:.3f}]".format(r_lower, r_upper))

Correlation r = 0.8703545613058227
Standard error (z scale) = 0.3333333333333333
95% CI for r: [0.592, 0.963]


In [2]:
import numpy as np
from statsmodels.stats.power import NormalIndPower
from scipy.stats import norm

# Function to compute power for correlation
def correlation_power(n, r_alt, alpha=0.05):
    """
    n       : sample size
    r_alt   : true correlation under alternative
    alpha   : significance level (two-tailed)
    """
    # Fisher z-transform
    z0 = 0  # null correlation r=0
    z1 = 0.5 * np.log((1 + r_alt) / (1 - r_alt))
    
    se = 1 / np.sqrt(n - 3)
    
    # Critical z-value for two-tailed test
    z_alpha = norm.ppf(1 - alpha/2)
    
    # Power calculation
    power = norm.cdf((z1 - z0)/se - z_alpha) + (1 - norm.cdf((z1 - z0)/se + z_alpha))
    return power

# Example: compute power for n=12 and r_alt=0.5
n = 12
r_alt = 0.5
alpha = 0.05
power = correlation_power(n, r_alt, alpha)
print(f"Statistical power for n={n}, r={r_alt}: {power:.3f}")

# Function to compute required sample size for desired power
def required_sample_size(r_alt, power_target=0.8, alpha=0.05):
    n = 4  # minimum starting sample size
    while True:
        if correlation_power(n, r_alt, alpha) >= power_target:
            return n
        n += 1

# Example: required sample size to detect r >= 0.5 with 80% power
n_required = required_sample_size(r_alt=0.5, power_target=0.8, alpha=0.05)
print(f"Required sample size for r>=0.5 with 80% power: {n_required}")

Statistical power for n=12, r=0.5: 0.378
Required sample size for r>=0.5 with 80% power: 30
